<a href="https://colab.research.google.com/github/programminghunk/sugarcane/blob/main/finalmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 36224, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 36224 (delta 74), reused 109 (delta 49), pack-reused 36071 (from 1)
Receiving objects: 100% (36224/36224), 29.32 MiB | 26.81 MiB/s, done.
Resolving deltas: 100% (26459/26459), done.


In [ ]:
#%cd ultralytics
#!pip install -e '.[dev]'

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5771.8/8062.4 GB disk)


In [ ]:
%cd ultralytics/cfg/models/v8

/kaggle/working/ultralytics/ultralytics/cfg/models/v8


In [ ]:
%%writefile newconf.yaml
# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv8 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 2 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 1, RepNCSPELAN4, [64, 64, 32, 3]] # 4
  - [-1, 1, Conv, [128,3,2]] # 5-P4/16
  - [-1, 1, RepNCSPELAN4, [128, 128, 48, 3]] # 6
  - [-1, 1, Conv, [256,3,2]] # 7-P5/32
  - [-1, 1, RepNCSPELAN4, [256, 256, 64, 3]] # 8
  - [-1, 1, SPPF, [256, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)

Writing newconf.yaml


In [ ]:
%cd ..

/kaggle/working


In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO("ultralytics/ultralytics/cfg/models/v8/newconf.yaml")

# Display model information (optional)
model.info()

WARNING ⚠️ no model scale passed. Assuming scale='n'.
newconf summary: 512 layers, 2,157,798 parameters, 2,157,782 gradients, 7.2 GFLOPs


(512, 2157798, 2157782, 7.17952)

In [ ]:
%cd ultralytics

/kaggle/working/ultralytics


In [ ]:
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("np_mynp")
version = project.version(2)
dataset = version.download("yolov8")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.76, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to np_mynp-2 in yolov8:: 100%|██████████| 6719/6719 [00:01<00:00, 5974.14it/s]


In [ ]:
%%writefile np_mynp-2/data.yaml
path: /kaggle/working/ultralytics
names:
- FIT
- UNFIT
nc: 2
roboflow:
  license: CC BY 4.0
  project: np_mynp
  url: https://universe.roboflow.com/buddetection/np_mynp/dataset/2
  version: 2
  workspace: buddetection
test: np_mynp-2/test/images
train: np_mynp-2/train/images
val: np_mynp-2/test/images


Overwriting np_mynp-2/data.yaml


In [ ]:
results = model.train(data="np_mynp-2/data.yaml", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/ultralytics/cfg/models/v8/newconf.yaml, data=np_mynp-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]
2024-08-13 16:26:40,668	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-13 16:26:41,986	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  1     58976  ultralytics.nn.modules.block.RepNCSPELAN4    [64, 16, 64, 32, 3]           
  5                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  6                  -1  1    130144  ultralytics.nn.modules.block.RepNCSPELAN4    [32, 32, 128, 48, 3]          
  7                  -1  1     18

100%|██████████| 6.25M/6.25M [00:00<00:00, 73.0MB/s]


AMP: checks passed ✅


...

    100/100      2.34G     0.7041     0.4479      1.112         22        640: 100%|██████████| 193/193 [00:35<00:00,  5.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.52it/s]

                   all        268        347      0.888      0.866      0.924      0.646



100 epochs completed in 1.124 hours.
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train/weights/last.pt, 4.7MB
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train/weights/best.pt, 4.7MB

Validating /kaggle/working/ultralytics/runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
newconf summary (fused): 297 layers, 2,140,422 parameters, 0 gradients, 7.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.57it/s]


                   all        268        347      0.894      0.884      0.928      0.649
                   FIT        125        162      0.902      0.849      0.921        0.7
                 UNFIT        163        185      0.885      0.919      0.934      0.599
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /kaggle/working/ultralytics/runs/detect/train


In [ ]:
%cd ..

/kaggle/working


In [ ]:
!zip -r file.zip /kaggle/working/ultralytics/runs/detect/train
from IPython.display import FileLink
FileLink(r'file.zip')

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


  adding: kaggle/working/ultralytics/runs/detect/train/ (stored 0%)
  adding: kaggle/working/ultralytics/runs/detect/train/confusion_matrix.png (deflated 37%)
  adding: kaggle/working/ultralytics/runs/detect/train/train_batch1.jpg (deflated 1%)
  adding: kaggle/working/ultralytics/runs/detect/train/labels.jpg (deflated 37%)
  adding: kaggle/working/ultralytics/runs/detect/train/R_curve.png (deflated 12%)
  adding: kaggle/working/ultralytics/runs/detect/train/val_batch1_labels.jpg (deflated 5%)
  adding: kaggle/working/ultralytics/runs/detect/train/weights/ (stored 0%)
  adding: kaggle/working/ultralytics/runs/detect/train/weights/best.pt (deflated 13%)
  adding: kaggle/working/ultralytics/runs/detect/train/weights/last.pt (deflated 13%)
  adding: kaggle/working/ultralytics/runs/detect/train/results.csv (deflated 85%)
  adding: kaggle/working/ultralytics/runs/detect/train/F1_curve.png (deflated 12%)
  adding: kaggle/working/ultralytics/runs/detect/train/val_batch2_labels.jpg (deflated 4

/kaggle/working/file.zip

In [ ]:
%pwd

'/kaggle/working'

In [ ]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model="ultralytics/runs/detect/train/weights/best.pt", data="data.yaml", imgsz=640, half=False, device=0)


image 1/1 /kaggle/working/ultralytics/ultralytics/assets/bus.jpg: 640x480 2 UNFITs, 84.7ms
Speed: 2.9ms preprocess, 84.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
ERROR ❌️ Benchmark failure for PyTorch: 'data.yaml' does not exist

PyTorch: starting from 'ultralytics/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (4.4 MB)

TorchScript: starting export with torch 2.1.2...
TorchScript: export success ✅ 2.8s, saved as 'ultralytics/runs/detect/train/weights/best.torchscript' (8.7 MB)

Export complete (2.8s)
Results saved to /kaggle/working/ultralytics/runs/detect/train/weights
Predict:         yolo predict task=detect model=ultralytics/runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=ultralytics/runs/detect/train/weights/best.torchscript imgsz=640 data=np_mynp-2/data.yaml  
Visualize:       https://netron.app
Loading ultralytics/runs/detect/train/weights/be

2024-08-12 08:57:26.751388965 [E:onnxruntime:Default, provider_bridge_ort.cc:1745 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1426 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2024-08-12 08:57:26.751429372 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:895 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirementsto ensure all dependencies are met.
2024-08-12 08:57:27.036539675 [E:onnxruntime:Default, provider_bridge_ort.cc:1745 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1426 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load lib

Setting batch=1 input of shape (1, 3, 640, 640)
ERROR ❌️ Benchmark failure for ONNX: 'data.yaml' does not exist
ERROR ❌️ Benchmark failure for OpenVINO: inference not supported on GPU

PyTorch: starting from 'ultralytics/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (4.4 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 1.4s, saved as 'ultralytics/runs/detect/train/weights/best.onnx' (8.3 MB)
requirements: Ultralytics requirement ['tensorrt>7.0.0,<=10.1.0'] not found, attempting AutoUpdate...
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tensorrt: filename=tensorrt-10.1.0-py2.py3-none-any.whl size=16333 sha256=0fb414d30bba0a7e894188280a4c7eec8e349f512f734909b23be01ccba7c2db
  Stored in directory: /tmp/pip-ephem-wheel-cache

100%|██████████| 1.11M/1.11M [00:00<00:00, 30.3MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /kaggle/working/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 44.34file/s]


ONNX: starting export with onnx 1.16.1 opset 17...


ONNX: slimming with onnxslim 0.1.32...
ONNX: export success ✅ 2.7s, saved as 'ultralytics/runs/detect/train/weights/best.onnx' (8.3 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.22.3...


In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO("ultralytics/ultralytics/cfg/models/v8/newconf.yaml")

# Display model information (optional)
model.info()
#results = model.train(data="coco128.yaml", epochs=100, imgsz=640)

WARNING ⚠️ no model scale passed. Assuming scale='n'.
newconf summary: 512 layers, 2,157,798 parameters, 2,157,782 gradients, 7.2 GFLOPs


(512, 2157798, 2157782, 7.17952)

In [ ]:
results = model.train(data="coco128.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.2.77 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/ultralytics/cfg/models/v8/newconf.yaml, data=coco128.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

train: Scanning /kaggle/working/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /kaggle/working/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


Plotting labels to /kaggle/working/ultralytics/runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 125 weight(decay=0.0), 132 weight(decay=0.0005), 131 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /kaggle/working/ultralytics/runs/detect/train6
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.76G       3.35      5.403      3.849         72        640: 100%|██████████| 8/8 [00:01<00:00,  4.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.82it/s]

                   all        128        929    0.00218     0.0222    0.00162    0.00043



100 epochs completed in 0.117 hours.
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train6/weights/last.pt, 5.0MB
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train6/weights/best.pt, 5.0MB

Validating /kaggle/working/ultralytics/runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
newconf summary (fused): 297 layers, 2,286,288 parameters, 0 gradients, 7.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.52it/s]


                   all        128        929     0.0147       0.02    0.00818    0.00177
                person         61        254    0.00377      0.165    0.00426   0.000938
               bicycle          3          6          0          0          0          0
                   car         12         46          0          0          0          0
            motorcycle          4          5    0.00376        0.2    0.00302   0.000302
              airplane          5          6          0          0          0          0
                   bus          5          7    0.00385      0.143     0.0024    0.00024
                 train          3          3          0          0          0          0
                 truck          5         12          0          0          0          0
                  boat          2          6          0          0          0          0
         traffic light          4         14          0          0          0          0
             stop sig

In [ ]:
%cd ..

/kaggle/working


In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO("ultralytics/ultralytics/cfg/models/v8/yolov8n.yaml")

# Display model information (optional)
model.info()
results = model.train(data="coco128.yaml", epochs=100, imgsz=640)

YOLOv8n summary: 225 layers, 3,157,200 parameters, 3,157,184 gradients, 8.9 GFLOPs
New https://pypi.org/project/ultralytics/8.2.77 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/ultralytics/cfg/models/v8/yolov8n.yaml, data=coco128.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fa

train: Scanning /kaggle/working/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /kaggle/working/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


Plotting labels to /kaggle/working/ultralytics/runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /kaggle/working/ultralytics/runs/detect/train7
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.59G      3.266      5.205      3.766         72        640: 100%|██████████| 8/8 [00:01<00:00,  5.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.69it/s]

                   all        128        929    0.00249     0.0581    0.00508    0.00138



100 epochs completed in 0.095 hours.
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train7/weights/last.pt, 6.5MB
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train7/weights/best.pt, 6.5MB

Validating /kaggle/working/ultralytics/runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.2.76 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]


                   all        128        929    0.00254     0.0553    0.00631    0.00176
                person         61        254    0.00401       0.28    0.00736    0.00198
               bicycle          3          6          0          0          0          0
                   car         12         46          0          0          0          0
            motorcycle          4          5    0.00371        0.8     0.0372    0.00872
              airplane          5          6     0.0112      0.167     0.0069    0.00207
                   bus          5          7    0.00395      0.143    0.00274    0.00137
                 train          3          3    0.00377      0.333    0.00272   0.000272
                 truck          5         12          0          0          0          0
                  boat          2          6          0          0          0          0
         traffic light          4         14          0          0          0          0
             stop sig